## Import Libraries

In [3]:
from vllm import LLM, SamplingParams
import tqdm as notebook_tqdm
import json
import os
import re
import copy
from huggingface_hub import login

with open('/home/nd2794/keys.json', 'r') as file:
    keys = json.load(file)

cache_dir = '/proj/vondrick4/naveen/HF_CACHE_DIR'
hf_token = keys["HF_READ_TOKEN"]
login(token=hf_token)

os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['PYTORCH_TRANSFORMERS_CACHE'] = cache_dir
os.environ['PYTORCH_PRETRAINED_BERT_CACHE'] = cache_dir
os.environ['HF_HUB_CACHE'] = cache_dir

/proj/vondrick2/naveen/miniconda3/envs/tr-vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-23 01:45:03,923	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Define Utility Functions

In [26]:
def get_instruction(source_event, target_event):
    common_instruct = """Two images (Source & Target) are described by their captions.
Your have generate a query text for composed image retrieval task that describes how the Source image should be modified to get the Target image.
Each image may multiple captions which are listed in a new line. You must read all the captions for each image to fully understand the image.
Only output the query text.
Note:
- The query text should describe only the difference.
- It is important that the query text should not fully or partially describe the source or target. 
- The query text must be short. Avoid writing long queries. 
- Avoid using trivial sentences like 'Repalce this with that'. Only describe the change.
- The query text must describe a subtle difference.
"""

    source_instruct = """SOURCE IMAGE CAPTIONS:\n""" + "\n".join(f"- {item}" for item in source_event)

    target_instruct = """TARGET IMAGE CAPTIONS:\n""" + "\n".join(f"- {item}" for item in target_event)

    return common_instruct + '\n\n' + source_instruct + '\n\n' + target_instruct

In [27]:
ex_source_cap = [
    'A person in blue and orange shirt riding a bicycle.',
    'A bicyclist riding in front of speed blurred foliage. ',
    'A cyclist with a helmet bicycles quickly on a road.',
    'The man is riding his bike down the street. '
]

ex_target_cap = [
    'A man is riding a motorcycle down the street',
    'A man riding a motorcycle down a street next to a restaurant.',
    'Helmeted motorcyclist riding on roadway in populated setting. ',
    'A motorcycle going down a street very fast.'
]

ex_query = "The person is riding a motorcycle"

In [28]:
conversation_template = [
    {
        "role": "system",
        "content": "You are a helpful assistant"
    },
    {
        "role": "user",
        "content": get_instruction(ex_source_cap, ex_target_cap)
    },
    {
        "role": "assistant",
        "content": ex_query 
    }
]

## Load Model

In [7]:
model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
llm = LLM(
    model=model_name, 
    tensor_parallel_size=2, 
    download_dir=cache_dir,
    dtype='bfloat16'
)

INFO 11-23 01:45:17 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-23 01:45:17 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-23 01:45:17 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-23 01:45:17 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir='/proj/vondrick4/naveen/HF_CACHE_DIR', load_format=LoadFormat.AUTO, tensor_parall

[W1123 01:45:20.353968595 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [localhost]:57367 (errno: 97 - Address family not supported by protocol).
[W1123 01:45:20.482064894 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [localhost]:57367 (errno: 97 - Address family not supported by protocol).


INFO 11-23 01:45:21 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/nd2794/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=2706752) WARNING 11-23 01:45:21 custom_all_reduce.py:141] Custom allreduce is disabled because your platform lacks GPU P2P capability or P2P test failed. To silence this warning, specify disable_custom_all_reduce=True explicitly.
INFO 11-23 01:45:21 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/nd2794/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=2706752) WARNING 11-23 01:45:21 custom_all_reduce.py:141] Custom allreduce is disabled because your platform lacks GPU P2P capability or P2P test failed. To silence this warning, specify disable_custom_all_reduce=True explicitly.
INFO 11-23 01:45:21 shm_broadcast.py:241] vLLM message queue communication handle: Handle(connect_ip='127.0.0.1', local_reader_ranks=[1], buffer=<vllm.distributed.device_communicators.shm_broadcast.ShmRin

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:30<01:31, 30.45s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [01:13<01:15, 37.88s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [01:23<00:25, 25.20s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [02:06<00:00, 32.35s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [02:06<00:00, 31.74s/it]



INFO 11-23 01:47:42 model_runner.py:1067] Loading model weights took 7.5122 GB
(VllmWorkerProcess pid=2706752) INFO 11-23 01:47:42 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-23 01:47:43 distributed_gpu_executor.py:57] # GPU blocks: 62613, # CPU blocks: 4096
INFO 11-23 01:47:43 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 7.64x
INFO 11-23 01:47:46 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-23 01:47:46 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=2706752) INFO 11-23 01:47:46 model_runner.py:1395] Capturing the mo

In [10]:
sampling_params = SamplingParams(
    #temperature=1.0,
    max_tokens=77,
)

## Load the data

In [29]:
split = 'train'
if split == 'train':
    data = json.load(open('/proj/vondrick4/naveen/coir-data/LaSCo/metadata/lasco_train_indexed_MT(img_cap).json', 'r'))
    imgs_and_caps = json.load(open('/proj/vondrick4/naveen/coir-data/annotations/images_and_caps_train2014_processed_dict.json', 'r'))
elif split == 'val':
    data = json.load(open('/proj/vondrick4/naveen/coir-data/LaSCo/metadata/lasco_val_indexed_MT(img_cap).json', 'r'))
    imgs_and_caps= json.load(open('/proj/vondrick4/naveen/coir-data/annotations/images_and_caps_val2014_processed_dict.json', 'r'))
    for key in imgs_and_caps.keys():
        imgs_and_caps[key] = [imgs_and_caps[key]]

## Process the data

In [30]:
conversations = []
tracker = {}
i = 0

for sample in data[:100]:
    tracker[sample['id']] = {}
    
    conversation_forward = copy.deepcopy(conversation_template)
    conversation_forward.append(
        {
            "role": "user",
            "content": get_instruction(imgs_and_caps[str(sample["coir"]["query-image-id"])], imgs_and_caps[str(sample["coir"]["target-image-id"])])
        }
    )
    
    conversation_reverse = copy.deepcopy(conversation_template)
    conversation_reverse.append(
        {
            "role": "user",
            "content": get_instruction(imgs_and_caps[str(sample["coir"]["target-image-id"])], imgs_and_caps[str(sample["coir"]["query-image-id"])])
        }
    )
    
    conversations.append(conversation_forward)
    tracker[sample['id']]['forward'] = i
    i+=1
    
    conversations.append(conversation_reverse)
    tracker[sample['id']]['reverse'] = i
    i+=1

## Run vLLM

In [31]:
outputs = llm.chat(
    messages=conversations,
    sampling_params=sampling_params,
    use_tqdm=True
)

Processed prompts: 100%|█████████████████████████████████████████████████████████████| 200/200 [00:07<00:00, 26.06it/s, est. speed input: 15395.26 toks/s, output: 252.65 toks/s]


## Process & Write Outputs

In [32]:
processed_outputs = []
for out in outputs:
    processed_outputs.append(out.outputs[0].text)

In [33]:
for sample in data[:100]:
    sample["coir"]['query-text-forward-mg'] = processed_outputs[tracker[sample['id']]['forward']]
    sample["coir"]['query-text-reverse-mg'] = processed_outputs[tracker[sample['id']]['reverse']]

In [34]:
with open('/proj/vondrick4/naveen/coir-data/LaSCo/metadata/lasco_{}_indexed_MT(img_cap)_MG_V2.json'.format(split), 'w') as json_file:
    json.dump(data, json_file, indent=4)

KeyboardInterrupt: 

In [35]:
data

[{'coir': {'query-image-id': 158307,
   'query-image-path': 'coco/train2014/COCO_train2014_000000158307.jpg',
   'target-image-id': 89462,
   'target-image-path': 'coco/train2014/COCO_train2014_000000089462.jpg',
   'query-text': 'Six people on the hill',
   'query-text-forward-mg': 'Add kites to the sky',
   'query-text-reverse-mg': 'Add a plane in the sky behind the kites'},
  'img_cap': {'image-id': 355737,
   'image-path': 'coco/train2014/COCO_train2014_000000355737.jpg',
   'caption': 'People walking down the street holding signs by a traffic light.'},
  'id': 0},
 {'coir': {'query-image-id': 472405,
   'query-image-path': 'coco/train2014/COCO_train2014_000000472405.jpg',
   'target-image-id': 79224,
   'target-image-path': 'coco/train2014/COCO_train2014_000000079224.jpg',
   'query-text': 'There are no crates in this picture',
   'query-text-forward-mg': 'Change the baskets to a fruit stand.',
   'query-text-reverse-mg': 'Convert loose fruits into bins or baskets'},
  'img_cap': 